# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [35]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [36]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/Weather_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Hermanus,ZA,2022-11-24 17:42:48,-34.4187,19.2345,60.80,56,88,10.27,overcast clouds
1,1,Mount Isa,AU,2022-11-24 17:42:48,-20.7333,139.5000,74.97,38,40,4.61,scattered clouds
2,2,Hambantota,LK,2022-11-24 17:42:49,6.1241,81.1185,76.77,85,80,8.52,broken clouds
3,3,Carnarvon,AU,2022-11-24 17:42:49,-24.8667,113.6333,68.16,75,0,20.71,clear sky
4,4,Souris,CA,2022-11-24 17:42:50,49.6172,-100.2612,26.46,86,96,9.93,overcast clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [37]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [38]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[
    (city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)
]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
1,1,Mount Isa,AU,2022-11-24 17:42:48,-20.7333,139.5000,74.97,38,40,4.61,scattered clouds
2,2,Hambantota,LK,2022-11-24 17:42:49,6.1241,81.1185,76.77,85,80,8.52,broken clouds
6,6,Honiara,SB,2022-11-24 17:42:51,-9.4333,159.9500,77.22,86,7,4.47,clear sky
7,7,Arraial Do Cabo,BR,2022-11-24 17:42:51,-22.9661,-42.0278,73.36,94,75,6.91,light rain
9,9,Castro,BR,2022-11-24 17:42:52,-24.7911,-50.0119,74.08,57,65,5.28,broken clouds
17,17,Malinyi,TZ,2022-11-24 17:42:55,-8.9333,36.1333,80.19,52,87,2.28,overcast clouds
19,19,Nanakuli,US,2022-11-24 17:42:56,21.3906,-158.1547,70.11,73,75,5.75,light rain
21,21,Victoria,HK,2022-11-24 17:40:52,22.2855,114.1577,72.18,92,100,11.01,overcast clouds
23,23,Sola,VU,2022-11-24 17:42:57,-13.8833,167.5500,81.63,71,60,7.83,broken clouds
24,24,Avarua,CK,2022-11-24 17:42:58,-21.2078,-159.7750,75.25,78,75,6.91,broken clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [42]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.count()
clean_travel_cities = preferred_cities_df.dropna()
clean_travel_cities.count()

City_ID        188
City           188
Country        188
Date           188
Lat            188
Lng            188
Max Temp       188
Humidity       188
Cloudiness     188
Wind Speed     188
Description    188
dtype: int64

## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [43]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Description", "Lat", "Lng"]].copy()


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [44]:
# Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Mount Isa,AU,74.97,scattered clouds,-20.7333,139.5000,
2,Hambantota,LK,76.77,broken clouds,6.1241,81.1185,
6,Honiara,SB,77.22,clear sky,-9.4333,159.9500,
7,Arraial Do Cabo,BR,73.36,light rain,-22.9661,-42.0278,
9,Castro,BR,74.08,broken clouds,-24.7911,-50.0119,
17,Malinyi,TZ,80.19,overcast clouds,-8.9333,36.1333,
19,Nanakuli,US,70.11,light rain,21.3906,-158.1547,
21,Victoria,HK,72.18,overcast clouds,22.2855,114.1577,
23,Sola,VU,81.63,broken clouds,-13.8833,167.5500,
24,Avarua,CK,75.25,broken clouds,-21.2078,-159.7750,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [45]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [46]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")



Starting hotel search
Mount Isa - nearest hotel: Ibis Styles
Hambantota - nearest hotel: Oasis Aurweda Resort Hambathota
Honiara - nearest hotel: King Solomon Hotel
Arraial Do Cabo - nearest hotel: No hotel found
Castro - nearest hotel: Hotel Central Palace
Malinyi - nearest hotel: No hotel found
Nanakuli - nearest hotel: No hotel found
Victoria - nearest hotel: 四季酒店 Four Seasons Hotel
Sola - nearest hotel: No hotel found
Avarua - nearest hotel: Paradise Inn
Vaini - nearest hotel: Keleti International Resort
Hilo - nearest hotel: Dolphin Bay Hotel
Bombay - nearest hotel: Hotel City Point
Rikitea - nearest hotel: Chez Bianca & Benoit
Luganville - nearest hotel: Beachfront Resort
Mayumba - nearest hotel: Résidence Hôtelière Fabso
Beira - nearest hotel: Hotel Savoy
Bondo - nearest hotel: No hotel found
Hithadhoo - nearest hotel: Pebbles Inn
Itarema - nearest hotel: Pousada Oásis
Georgetown - nearest hotel: Page 63 hostel
Bambanglipuro - nearest hotel: No hotel found
Alugan - nearest hotel

In [47]:
# Display sample data
hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Mount Isa,AU,74.97,scattered clouds,-20.7333,139.5000,Ibis Styles
2,Hambantota,LK,76.77,broken clouds,6.1241,81.1185,Oasis Aurweda Resort Hambathota
6,Honiara,SB,77.22,clear sky,-9.4333,159.9500,King Solomon Hotel
7,Arraial Do Cabo,BR,73.36,light rain,-22.9661,-42.0278,No hotel found
9,Castro,BR,74.08,broken clouds,-24.7911,-50.0119,Hotel Central Palace
...,...,...,...,...,...,...,...
425,Japoata,BR,79.95,clear sky,-10.3467,-36.8011,No hotel found
426,Djibo,BF,80.74,overcast clouds,14.1022,-1.6306,No hotel found
429,Kuching,MY,77.22,few clouds,1.5500,110.3333,Hotel Samudera
431,Bushehr,IR,73.49,clear sky,28.9684,50.8385,هتل رستوران سیراف


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [48]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.copy()
rows_to_drop = hotel_df[hotel_df["Hotel Name"]=="No hotel found"].index
clean_hotel_df.drop(rows_to_drop, inplace = True)
# Display sample data
clean_hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Mount Isa,AU,74.97,scattered clouds,-20.7333,139.5000,Ibis Styles
2,Hambantota,LK,76.77,broken clouds,6.1241,81.1185,Oasis Aurweda Resort Hambathota
6,Honiara,SB,77.22,clear sky,-9.4333,159.9500,King Solomon Hotel
9,Castro,BR,74.08,broken clouds,-24.7911,-50.0119,Hotel Central Palace
21,Victoria,HK,72.18,overcast clouds,22.2855,114.1577,四季酒店 Four Seasons Hotel
...,...,...,...,...,...,...,...
420,Mackay,AU,70.68,overcast clouds,-21.1500,149.2000,Mackay Oceanside Central Apartment Hotel
423,Saint-Leu,RE,74.23,clear sky,-21.1500,55.2833,Iloha Seaview Hôtel
429,Kuching,MY,77.22,few clouds,1.5500,110.3333,Hotel Samudera
431,Bushehr,IR,73.49,clear sky,28.9684,50.8385,هتل رستوران سیراف


In [50]:
clean_hotel_df = clean_hotel_df.reset_index(drop = True)
clean_hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Mount Isa,AU,74.97,scattered clouds,-20.7333,139.5000,Ibis Styles
1,Hambantota,LK,76.77,broken clouds,6.1241,81.1185,Oasis Aurweda Resort Hambathota
2,Honiara,SB,77.22,clear sky,-9.4333,159.9500,King Solomon Hotel
3,Castro,BR,74.08,broken clouds,-24.7911,-50.0119,Hotel Central Palace
4,Victoria,HK,72.18,overcast clouds,22.2855,114.1577,四季酒店 Four Seasons Hotel
...,...,...,...,...,...,...,...
123,Mackay,AU,70.68,overcast clouds,-21.1500,149.2000,Mackay Oceanside Central Apartment Hotel
124,Saint-Leu,RE,74.23,clear sky,-21.1500,55.2833,Iloha Seaview Hôtel
125,Kuching,MY,77.22,few clouds,1.5500,110.3333,Hotel Samudera
126,Bushehr,IR,73.49,clear sky,28.9684,50.8385,هتل رستوران سیراف


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [51]:
# Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file)

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [60]:
# Configure the map plot
map_plot_1 = clean_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Country", "Hotel Name", "Description"]
    
)
# Display the map plot
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Country,Hotel Name,Description)